<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_AdvanceDL_Project/blob/main/models/option_model_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import os
import pandas as pd
import numpy as np
from google.colab import drive
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
option = pd.read_csv('/gdrive/My Drive/Data set/Option/WIX_call-options-black-scholes.csv')
underlying = pd.read_csv('/gdrive/My Drive/Data set/Option/WIX_underlying.csv')

In [ ]:
underlying

,DataDate,UnderlyingPrice,sigma_5,sigma_10,sigma_21,sigma_30
0,1/2/2018 04:00:00 PM,10.98,NaN,NaN,NaN,NaN
1,1/3/2018 04:00:00 PM,11.55,NaN,NaN,NaN,NaN
2,1/4/2018 04:00:00 PM,12.12,NaN,NaN,NaN,NaN
3,1/5/2018 04:00:00 PM,11.88,NaN,NaN,NaN,NaN
4,1/8/2018 04:00:00 PM,12.28,0.033379,NaN,NaN,NaN
...,...,...,...,...,...,...
391,7/25/2019 04:00:00 PM,33.67,0.015078,0.019606,0.018441,0.022449
392,7/26/2019 04:00:00 PM,34.02,0.015076,0.015556,0.017786,0.021421
393,7/29/2019 04:00:00 PM,33.48,0.017021,0.015576,0.018002,0.020211
394,7/30/2019 04:00:00 PM,33.87,0.014728,0.015906,0.017308,0.018911


In [ ]:
option.head()

,Unnamed: 0,Unnamed: 0.1,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionSymbol,Blank,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega,Alias,sigma_5,sigma_10,sigma_21,sigma_30,date_diff,treasury_rate,black_scholes_pred_21
0,0,0,WIX,57.95,*,WIX180119C00040000,NaN,01/19/2018,1/2/2018 04:00:00 PM,40.0,17.90,17.00,20.30,0.0,12.0,2.0474,0.8556,0.8885,-17.1540,2.8402,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
1,2,2,WIX,57.95,*,WIX180119C00045000,NaN,01/19/2018,1/2/2018 04:00:00 PM,45.0,14.82,11.70,15.40,0.0,14.0,1.6263,0.8151,1.3130,-15.9985,3.3340,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
2,4,4,WIX,57.95,*,WIX180119C00050000,NaN,01/19/2018,1/2/2018 04:00:00 PM,50.0,9.20,7.70,9.60,0.0,24.0,0.9858,0.7884,2.3499,-10.5303,3.6168,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
3,6,6,WIX,57.95,*,WIX180119C00055000,NaN,01/19/2018,1/2/2018 04:00:00 PM,55.0,3.61,3.60,4.30,0.0,314.0,0.5262,0.6978,5.3048,-6.7828,4.3585,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
4,8,8,WIX,57.95,*,WIX180119C00060000,NaN,01/19/2018,1/2/2018 04:00:00 PM,60.0,1.45,1.15,1.65,181.0,726.0,0.5014,0.3950,6.1458,-7.1221,4.8110,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN


In [ ]:
option

,Unnamed: 0,Unnamed: 0.1,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionSymbol,Blank,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega,Alias,sigma_5,sigma_10,sigma_21,sigma_30,date_diff,treasury_rate,black_scholes_pred_21
0,0,0,WIX,57.95,*,WIX180119C00040000,NaN,01/19/2018,1/2/2018 04:00:00 PM,40.0,17.90,17.00,20.30,0.0,12.0,2.0474,0.8556,0.8885,-17.1540,2.8402,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
1,2,2,WIX,57.95,*,WIX180119C00045000,NaN,01/19/2018,1/2/2018 04:00:00 PM,45.0,14.82,11.70,15.40,0.0,14.0,1.6263,0.8151,1.3130,-15.9985,3.3340,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
2,4,4,WIX,57.95,*,WIX180119C00050000,NaN,01/19/2018,1/2/2018 04:00:00 PM,50.0,9.20,7.70,9.60,0.0,24.0,0.9858,0.7884,2.3499,-10.5303,3.6168,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
3,6,6,WIX,57.95,*,WIX180119C00055000,NaN,01/19/2018,1/2/2018 04:00:00 PM,55.0,3.61,3.60,4.30,0.0,314.0,0.5262,0.6978,5.3048,-6.7828,4.3585,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
4,8,8,WIX,57.95,*,WIX180119C00060000,NaN,01/19/2018,1/2/2018 04:00:00 PM,60.0,1.45,1.15,1.65,181.0,726.0,0.5014,0.3950,6.1458,-7.1221,4.8110,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51487,103562,103562,WIX,148.52,*,WIX210115C00185000,NaN,01/15/2021,7/31/2019 04:00:00 PM,185.0,0.00,16.40,19.20,0.0,0.0,0.4199,0.4462,0.5241,-2.9859,71.0047,NaN,0.053055,0.037583,0.02923,0.027146,534,2.00,3.659971e-08
51488,103564,103564,WIX,148.52,*,WIX210115C00190000,NaN,01/15/2021,7/31/2019 04:00:00 PM,190.0,15.33,15.00,17.80,0.0,4.0,0.4175,0.4239,0.5223,-2.9365,70.3513,NaN,0.053055,0.037583,0.02923,0.027146,534,2.00,3.771637e-10
51489,103566,103566,WIX,148.52,*,WIX210115C00195000,NaN,01/15/2021,7/31/2019 04:00:00 PM,195.0,13.40,13.90,16.40,0.0,1.0,0.4139,0.4014,0.5201,-2.8700,69.4568,NaN,0.053055,0.037583,0.02923,0.027146,534,2.00,2.594149e-12
51490,103568,103568,WIX,148.52,*,WIX210115C00200000,NaN,01/15/2021,7/31/2019 04:00:00 PM,200.0,11.80,12.90,15.20,0.0,16.0,0.4120,0.3805,0.5147,-2.8101,68.4214,NaN,0.053055,0.037583,0.02923,0.027146,534,2.00,1.226700e-14


In [ ]:
option.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51492 entries, 0 to 51491
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             51492 non-null  int64  
 1   Unnamed: 0.1           51492 non-null  int64  
 2   UnderlyingSymbol       51492 non-null  object 
 3   UnderlyingPrice        51492 non-null  float64
 4   Exchange               51492 non-null  object 
 5   OptionSymbol           51492 non-null  object 
 6   Blank                  0 non-null      float64
 7   Expiration             51492 non-null  object 
 8   DataDate               51492 non-null  object 
 9   Strike                 51492 non-null  float64
 10  Last                   51492 non-null  float64
 11  Bid                    51492 non-null  float64
 12  Ask                    51492 non-null  float64
 13  Volume                 51492 non-null  float64
 14  OpenInterest           51492 non-null  float64
 15  IV

In [ ]:
call_df = option

In [ ]:
N_TIMESTEPS = 21
padded = np.insert(underlying.UnderlyingPrice.values, 0, np.array([np.nan] * N_TIMESTEPS))
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]
rolled = np.column_stack((underlying.DataDate.values[N_TIMESTEPS - 1:], rolled))
price_history = pd.DataFrame(data=rolled)
joined = option.join(price_history.set_index(0), on='DataDate')

In [ ]:
joined.shape

(51492, 49)

In [ ]:
joined.head()

,Unnamed: 0,Unnamed: 0.1,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionSymbol,Blank,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega,Alias,sigma_5,sigma_10,sigma_21,sigma_30,date_diff,treasury_rate,black_scholes_pred_21,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,0,0,WIX,57.95,*,WIX180119C00040000,NaN,01/19/2018,1/2/2018 04:00:00 PM,40.0,17.90,17.00,20.30,0.0,12.0,2.0474,0.8556,0.8885,-17.1540,2.8402,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,WIX,57.95,*,WIX180119C00045000,NaN,01/19/2018,1/2/2018 04:00:00 PM,45.0,14.82,11.70,15.40,0.0,14.0,1.6263,0.8151,1.3130,-15.9985,3.3340,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,4,WIX,57.95,*,WIX180119C00050000,NaN,01/19/2018,1/2/2018 04:00:00 PM,50.0,9.20,7.70,9.60,0.0,24.0,0.9858,0.7884,2.3499,-10.5303,3.6168,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,6,WIX,57.95,*,WIX180119C00055000,NaN,01/19/2018,1/2/2018 04:00:00 PM,55.0,3.61,3.60,4.30,0.0,314.0,0.5262,0.6978,5.3048,-6.7828,4.3585,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,8,WIX,57.95,*,WIX180119C00060000,NaN,01/19/2018,1/2/2018 04:00:00 PM,60.0,1.45,1.15,1.65,181.0,726.0,0.5014,0.3950,6.1458,-7.1221,4.8110,NaN,NaN,NaN,NaN,NaN,17,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
option = joined

In [ ]:
option = option.drop(columns=['Unnamed: 0','Unnamed: 0.1','UnderlyingSymbol','Exchange','OptionSymbol','Blank','Expiration',
                     'DataDate','Last','Volume','OpenInterest','IV','Delta','Gamma','Theta','Vega','Alias',
                     'sigma_5','sigma_10','sigma_21','sigma_30'])

In [ ]:
option = option.drop(columns=['black_scholes_pred_21'])

In [ ]:
call_df = option

In [ ]:
call_df = call_df.dropna()

In [ ]:
call_df

,UnderlyingPrice,Strike,Bid,Ask,date_diff,treasury_rate,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
1659,61.05,30.0,30.3,31.8,16,1.43,13.735,12.865,13.32,12.95,12.41,12.71,12.94,12.64,12.59,12.47,12.18,11.91,12.02,12.1404,11.96,11.82,12.28,11.88,12.12,11.55,10.98
1660,61.05,35.0,25.3,26.9,16,1.43,13.735,12.865,13.32,12.95,12.41,12.71,12.94,12.64,12.59,12.47,12.18,11.91,12.02,12.1404,11.96,11.82,12.28,11.88,12.12,11.55,10.98
1661,61.05,40.0,20.4,22.0,16,1.43,13.735,12.865,13.32,12.95,12.41,12.71,12.94,12.64,12.59,12.47,12.18,11.91,12.02,12.1404,11.96,11.82,12.28,11.88,12.12,11.55,10.98
1662,61.05,45.0,15.3,16.7,16,1.43,13.735,12.865,13.32,12.95,12.41,12.71,12.94,12.64,12.59,12.47,12.18,11.91,12.02,12.1404,11.96,11.82,12.28,11.88,12.12,11.55,10.98
1663,61.05,50.0,10.8,12.2,16,1.43,13.735,12.865,13.32,12.95,12.41,12.71,12.94,12.64,12.59,12.47,12.18,11.91,12.02,12.1404,11.96,11.82,12.28,11.88,12.12,11.55,10.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51487,148.52,185.0,16.4,19.2,534,2.00,30.45,33.87,33.48,34.02,33.67,34.11,33.49,32.85,32.51,33,33.6,33.85,34.39,33.21,33.06,33.79,33.15,32.04,31.5,31.19,31.24
51488,148.52,190.0,15.0,17.8,534,2.00,30.45,33.87,33.48,34.02,33.67,34.11,33.49,32.85,32.51,33,33.6,33.85,34.39,33.21,33.06,33.79,33.15,32.04,31.5,31.19,31.24
51489,148.52,195.0,13.9,16.4,534,2.00,30.45,33.87,33.48,34.02,33.67,34.11,33.49,32.85,32.51,33,33.6,33.85,34.39,33.21,33.06,33.79,33.15,32.04,31.5,31.19,31.24
51490,148.52,200.0,12.9,15.2,534,2.00,30.45,33.87,33.48,34.02,33.67,34.11,33.49,32.85,32.51,33,33.6,33.85,34.39,33.21,33.06,33.79,33.15,32.04,31.5,31.19,31.24


In [ ]:
call_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49833 entries, 1659 to 51491
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   UnderlyingPrice  49833 non-null  float64
 1   Strike           49833 non-null  float64
 2   Bid              49833 non-null  float64
 3   Ask              49833 non-null  float64
 4   date_diff        49833 non-null  int64  
 5   treasury_rate    49833 non-null  float64
 6   1                49833 non-null  object 
 7   2                49833 non-null  object 
 8   3                49833 non-null  object 
 9   4                49833 non-null  object 
 10  5                49833 non-null  object 
 11  6                49833 non-null  object 
 12  7                49833 non-null  object 
 13  8                49833 non-null  object 
 14  9                49833 non-null  object 
 15  10               49833 non-null  object 
 16  11               49833 non-null  object 
 17  12       

In [ ]:
for i in range(1,22):
  call_df[i] = call_df[i].astype('float64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
call_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49833 entries, 1659 to 51491
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   UnderlyingPrice  49833 non-null  float64
 1   Strike           49833 non-null  float64
 2   Bid              49833 non-null  float64
 3   Ask              49833 non-null  float64
 4   date_diff        49833 non-null  int64  
 5   treasury_rate    49833 non-null  float64
 6   1                49833 non-null  float64
 7   2                49833 non-null  float64
 8   3                49833 non-null  float64
 9   4                49833 non-null  float64
 10  5                49833 non-null  float64
 11  6                49833 non-null  float64
 12  7                49833 non-null  float64
 13  8                49833 non-null  float64
 14  9                49833 non-null  float64
 15  10               49833 non-null  float64
 16  11               49833 non-null  float64
 17  12       

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['Bid', 'Ask'], axis=1).values,
                                                                        ((call_df.Bid + call_df.Ask) / 2).values,
                                                                        test_size=0.2, random_state=42)

In [ ]:
call_X_train.shape

(39866, 25)

In [ ]:
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]

In [ ]:
layers = 4
n_timesteps = 60
features = 4
n_batch = 256
n_epochs = 100

In [ ]:

def make_model():
    close_history = Input((N_TIMESTEPS, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(N_TIMESTEPS, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [ ]:
call_model = make_model()

In [ ]:
call_model.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 21, 1)]      0                                            
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 8)            1952        input_13[0][0]                   
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 12)           0           sequential_6[0][0]               
                                                                 input_14[0][0]       

In [ ]:
call_model.compile(optimizer=Adam(lr=1e-3), loss='mse')
history = call_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=400, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1
                    )

Epoch 1/400
155/155 [==============================] - 7s 45ms/step - loss: 2.2339 - val_loss: 0.5369
Epoch 2/400
155/155 [==============================] - 6s 37ms/step - loss: 2.2536 - val_loss: 0.5767
Epoch 3/400
155/155 [==============================] - 6s 38ms/step - loss: 2.2771 - val_loss: 1.1630
Epoch 4/400
155/155 [==============================] - 6s 38ms/step - loss: 2.0542 - val_loss: 0.9588
Epoch 5/400
155/155 [==============================] - 6s 37ms/step - loss: 2.1497 - val_loss: 0.8562
Epoch 6/400
155/155 [==============================] - 6s 38ms/step - loss: 2.0890 - val_loss: 0.7889
Epoch 7/400
155/155 [==============================] - 6s 39ms/step - loss: 1.9059 - val_loss: 0.6526
Epoch 8/400
155/155 [==============================] - 6s 39ms/step - loss: 1.9993 - val_loss: 0.5785
Epoch 9/400
155/155 [==============================] - 6s 38ms/step - loss: 1.9994 - val_loss: 0.4241
Epoch 10/400
155/155 [==============================] - 6s 39ms/step - loss: 2.147

In [ ]:
call_model.evaluate(call_X_train, call_y_train, batch_size=4096)

10/10 [==============================] - 1s 81ms/step - loss: 2.1128


2.1128334999084473